In [1]:
#Libraries
import pandas as pd
import numpy as np
import ast
import re
import spacy
from scispacy.abbreviation import AbbreviationDetector
from nltk.stem import WordNetLemmatizer 
import itertools
from nltk import word_tokenize 
from nltk.util import ngrams
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
df = pd.read_excel('C:/Users/GOWRICH1/Downloads/HAWK AI/Sprints/sprint4/Dosage_Cleaning/new_molecules_cleaned.xlsx')
df.head()

XML  \
0  2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1  b979da00-7c42-457e-b0d4-900c4844459c.xml   
2  ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3  53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4  d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   

                                               Title  \
0  Review article: anti-adhesion therapies for in...   
1  Treatment options for patients with multiple s...   
2  Intravenous immunoglobulin for maintenance tre...   
3  Natalizumab improves ambulation in relapsing&#...   
4  Application of Quantitative Pharmacology Appro...   

                                      Full_body_text  \
0  Introduction  Inflammatory bowel diseases (IBD...   
1  Introduction  Interferon beta (IFN β ) and gla...   
2  INTRODUCTION  Multifocal motor neuropathy (MMN...   
3  Introduction  Mobility problems are reported i...   
4  Domagrozumab is a human IgG1 monoclonal antibo...   

                                       Abstract_full                DOI  \
0  {'Abstract': 'A high proportion of patients wi...  10.1111/apt.12639   
1  {'Abstract': 'Although the first‐line disease‐...  10.1111/ene.12299   
2  {'Abstract': 'Intravenous immunoglobulin (IVIg...  10.1111/jns.12268   
3  {'Abstract': 'Impaired ambulation is a promine...  10.1111/ene.12618   
4  {'Abstract': 'Domagrozumab, a monoclonal antib...  10.1002/jcph.1015   

                                 URL     Molecule Publisher_type  \
0  https://doi.org/10.1111/apt.12639  Natalizumab          Wiley   
1  https://doi.org/10.1111/ene.12299  Natalizumab          Wiley   
2  https://doi.org/10.1111/jns.12268  Natalizumab          Wiley   
3  https://doi.org/10.1111/ene.12618  Natalizumab          Wiley   
4  https://doi.org/10.1002/jcph.1015  Natalizumab          Wiley   

                                       Molecule_text  \
0  ['we performed a literature review in pubmed a...   
1  ['natalizumab and fingolimod are approved for ...   
2  ['the main exclusion criteria were receiving (...   
3  ['whether natalizumab, a monoclonal antibody s...   
4  ['estimated clearance for various monoclonal a...   

                                        Intervention  ...  ABR SGRQ  ACQ FEV1  \
0  showed that intravenous (iv) infusion of natal...  ...  NaN  NaN  NaN  NaN   
1  in contrast fingolimod 0.5mg is a smaller mole...  ...  NaN  NaN  NaN  NaN   
2  the main exclusion criteria were receiving (1)...  ...  NaN  NaN  NaN  NaN   
3  timer: prospective analysis   participants and...  ...  NaN  NaN  NaN  NaN   
4  estimated clearance for various monoclonal ant...  ...  NaN  NaN  NaN  NaN   

  NEDA AQLQ AQoL                               Intervention_cleaned  \
0  NaN  NaN  NaN  showed that intravenous (iv) infusion of natal...   
1  NaN  NaN  NaN  in contrast fingolimod 0.5mg is a smaller mole...   
2  NaN  NaN  NaN  the main exclusion criteria were receiving (1)...   
3  NaN  NaN  NaN  timer: prospective analysis participants and d...   
4  NaN  NaN  NaN  estimated clearance for various monoclonal ant...   

  Dosage_clean PHASES21  
0     ['3 mg']  Phase 3  
1   ['0.5 mg']      NaN  
2      ['1 g']  Phase 3  
3   ['300 mg']      NaN  
4     ['2 mg']      NaN  

[5 rows x 96 columns]

In [7]:
df=df.replace(np.nan,"")
df.head()

XML  \
0  2da20cd7-0125-4728-a219-7cde1329c3e0.xml   
1  b979da00-7c42-457e-b0d4-900c4844459c.xml   
2  ecd677c9-8722-4314-aa15-9c956b38bbd3.xml   
3  53c91dea-5b3c-48fe-ba40-536c77e50012.xml   
4  d1a3a84e-d66f-439c-9874-3c3fdfaf6403.xml   

                                               Title  \
0  Review article: anti-adhesion therapies for in...   
1  Treatment options for patients with multiple s...   
2  Intravenous immunoglobulin for maintenance tre...   
3  Natalizumab improves ambulation in relapsing&#...   
4  Application of Quantitative Pharmacology Appro...   

                                      Full_body_text  \
0  Introduction  Inflammatory bowel diseases (IBD...   
1  Introduction  Interferon beta (IFN β ) and gla...   
2  INTRODUCTION  Multifocal motor neuropathy (MMN...   
3  Introduction  Mobility problems are reported i...   
4  Domagrozumab is a human IgG1 monoclonal antibo...   

                                       Abstract_full                DOI  \
0  {'Abstract': 'A high proportion of patients wi...  10.1111/apt.12639   
1  {'Abstract': 'Although the first‐line disease‐...  10.1111/ene.12299   
2  {'Abstract': 'Intravenous immunoglobulin (IVIg...  10.1111/jns.12268   
3  {'Abstract': 'Impaired ambulation is a promine...  10.1111/ene.12618   
4  {'Abstract': 'Domagrozumab, a monoclonal antib...  10.1002/jcph.1015   

                                 URL     Molecule Publisher_type  \
0  https://doi.org/10.1111/apt.12639  Natalizumab          Wiley   
1  https://doi.org/10.1111/ene.12299  Natalizumab          Wiley   
2  https://doi.org/10.1111/jns.12268  Natalizumab          Wiley   
3  https://doi.org/10.1111/ene.12618  Natalizumab          Wiley   
4  https://doi.org/10.1002/jcph.1015  Natalizumab          Wiley   

                                       Molecule_text  \
0  ['we performed a literature review in pubmed a...   
1  ['natalizumab and fingolimod are approved for ...   
2  ['the main exclusion criteria were receiving (...   
3  ['whether natalizumab, a monoclonal antibody s...   
4  ['estimated clearance for various monoclonal a...   

                                        Intervention  ... ABR SGRQ ACQ FEV1  \
0  showed that intravenous (iv) infusion of natal...  ...                     
1  in contrast fingolimod 0.5mg is a smaller mole...  ...                     
2  the main exclusion criteria were receiving (1)...  ...                     
3  timer: prospective analysis   participants and...  ...                     
4  estimated clearance for various monoclonal ant...  ...                     

  NEDA AQLQ AQoL                               Intervention_cleaned  \
0                 showed that intravenous (iv) infusion of natal...   
1                 in contrast fingolimod 0.5mg is a smaller mole...   
2                 the main exclusion criteria were receiving (1)...   
3                 timer: prospective analysis participants and d...   
4                 estimated clearance for various monoclonal ant...   

  Dosage_clean PHASES21  
0     ['3 mg']  Phase 3  
1   ['0.5 mg']           
2      ['1 g']  Phase 3  
3   ['300 mg']           
4     ['2 mg']           

[5 rows x 96 columns]

### Identifying Abbreviations 

In [ ]:
nlp = spacy.load("en_core_sci_sm")

In [ ]:
nlp.add_pipe("abbreviation_detector")

In [ ]:
doc=nlp(', '.join((df['competing_trials'].tolist())).lower().replace("-"," "))

In [ ]:
sa=', '.join((df['competing_trials'].tolist())).lower().replace("-"," ")
sa

In [ ]:
abrv_list=[]
abrv_form_list=[]
for abrv in doc._.abbreviations:
    #print(abrv)
    if len(str(abrv._.long_form).split(" "))>1:
        if str(abrv) not in abrv_list :
            abrv_list.append(str(abrv))
        if(abrv._.long_form) not in abrv_form_list:
            abrv_form_list.append(abrv._.long_form)
        sa=sa.replace(str(abrv),str(abrv._.long_form))
        sa=sa.replace("("+str(abrv._.long_form)+")","")

In [ ]:
abrv_list

In [ ]:
abrv_form_list

In [ ]:
#Remove or add or modify any dictionary items

abbr_dict = dict(zip(abrv_list, abrv_form_list))
abbr_dict

In [ ]:
sa=', '.join((df['competing_trials'].tolist())).lower().replace("-"," ")
sa

In [ ]:
abbr_dis_list=[]
for w in (sa.split(", ")):
    w = re.sub("[\(].*?[\)]", "", w)
    w = re.sub(" +"," ",w)
    for wo in w.split(" "):
        #wo = re.sub("[\(].*?[\)]", "", wo)
        #wo = re.sub(" +"," ",wo)
        if wo in abbr_dict.keys():
            w = w.replace(wo,str(abbr_dict.get(wo)))
    abbr_dis_list.append(w)

### To get frequent Bigrams

In [ ]:
class Seeder:
    def __init__(self):
        self.seeds = set()
        self.cache = dict()

    def get_seed(self, word):
        LIMIT = 2
        seed = self.cache.get(word,None)
        if seed is not None:
            return seed
        for seed in self.seeds:
            if self.distance(seed, word) <= LIMIT:
                self.cache[word] = seed
                return seed
        self.seeds.add(word)
        self.cache[word] = word
        return word

    def distance(self, s1, s2):
        l1 = len(s1)
        l2 = len(s2)
        matrix = [list(range(zz,zz + l1 + 1)) for zz in list(range(l2 + 1))]
        for zz in list(range(0,l2)):
            for sz in list(range(0,l1)):
                if s1[sz] == s2[zz]:
                    matrix[zz+1][sz+1] = min(matrix[zz+1][sz] + 1, matrix[zz][sz+1] + 1, matrix[zz][sz])
                else:
                    matrix[zz+1][sz+1] = min(matrix[zz+1][sz] + 1, matrix[zz][sz+1] + 1, matrix[zz][sz] + 1)
        return matrix[l2][l1]



def group_similar(words):
    seeder = Seeder()
    words = sorted(words, key=seeder.get_seed)
    groups = itertools.groupby(words, key=seeder.get_seed)
    return [list(v) for k,v in groups]

In [ ]:
elem=group_similar(abbr_dis_list)
elem

In [ ]:
c=0
text = elem
total_list = []
for line in text:
    for d in line:
#         print(data)
        d = str(d)
        token = word_tokenize(d)
        bigram = list(ngrams(token, 2)) 
        c=c+1
        for l in bigram:
            list1=['(',')','or','and','of','stage','for','cell','with','in','non','the','after']
            if any(x in list1 for x in l):
                pass
            else:
                if l[0].isalpha() and l[1].isalpha() and len(l[0])>2 and len(l[1])>2:
                    if not l[0].startswith("ii"):
                        if not l[1].startswith("ii"):
                            total_list.append(' '.join(l))
                else:
                    pass
print(total_list)

### Bigram filtering

In [ ]:
import spacy
nlp3 = spacy.load("en_ner_bc5cdr_md")#DISEASE and CHEMICAL

In [ ]:
draft_list=[]
for i in total_list:
    doc = nlp3(i)
    c=[ent.text.lower() for ent in doc.ents if ent.label_=='DISEASE' and ent.text.lower()not in ("hi","please","nt","s","im","of") 
       if not ent.text.startswith("@")if not ent.text.startswith("http")
       if not ent.text.startswith("#")if not ent.text.startswith("https")
      ]
    if c!='':
        for t in c:
            draft_list.append(t)

In [ ]:
#final Bigram data to compare
draft_data = (list(set(filter(None, draft_list))))
draft_data

In [ ]:
def get_abbreviation(disease_value):
    input_list=[]
    for w in (disease_value.split(", ")):
        w = re.sub("[\(].*?[\)]", "", w)
        w = re.sub(" +"," ",w)
        for wo in w.split(" "):
            #wo = re.sub("[\(].*?[\)]", "", wo)
            #wo = re.sub(" +"," ",wo)
            if wo.lower() in abbr_dict.keys():
                w = w.replace(wo,str(abbr_dict.get(wo.lower())))
        input_list.append(w.lower())
    return input_list

In [ ]:
lemmatizer = WordNetLemmatizer()
def clean_disease1(diseases):
    dis_list=[]
    try:
        for a in diseases:
            #print(a)
            word=""
            for b in a.split(" "):
                #print(b)
                #if not b.startswith("atop"):
                    #c = str(TextBlob(b).correct())  #Spelling correction
                b = re.sub("[\(].*?[\)]", "", b)                
                word = word + lemmatizer.lemmatize(b)+" " #Singular words
                #else:
                    #word = word + lemmatizer.lemmatize(b)+" "
                #print(word)
            word = word.rstrip(' ') 
            word = re.sub('\s+',' ',word)
            
            #print(word)
            if word not in ('na','NA'):
                if len(word)<5:
                    dis_list.append(word.replace("-"," ")) 
                else:
                    dis_list.append(word.lower().replace("tumour","tumor").replace("-"," "))
        return ','.join(dis_list) 
    except:
        pass

### Cleaning and Mapping diseases 

In [ ]:
df['DISEASE_filtered']=''

In [ ]:
for i in range(0,len(df['competing_trials'])):
    print(i)
    final_dis_list=[]
    other_dis_list=[]
    try:
        dis_list=clean_disease(get_abbreviation(df['competing_trials'].iloc[i]))
        dis_exclusion_list=['and','disease','infection']
        other_list=dis_list.split(",")
        
        for w in other_list: 
            
            if (len(w.split())<2) or  (any(x in w for x in dis_exclusion_list)):
                    final_dis_list.append(w)            
            else:
                try:
                    #dis = process.extractOne(w,draft_data,score_cutoff=80)[0]
                    dis_l = process.extract(w,draft_data,limit=2)
                    dis = dis_l[0][0]
                    dis1 = dis_l[1][0]
                    if len(dis.split(" "))==1:                        
                        if len(dis1.split(" "))>1:
                            if fuzz.token_set_ratio(w,dis1)>90:
                                final_dis_list.append(dis1)
                            else:
                                final_dis_list.append(w)
                        else:
                            if fuzz.token_set_ratio(w,dis)>90:
                                final_dis_list.append(dis)
                            else:
                                final_dis_list.append(w)
                    else:
                        if fuzz.token_set_ratio(w,dis)>90:
                                final_dis_list.append(dis)
                        else:
                            final_dis_list.append(w)
                except:
                    final_dis_list.append(w)
                    
        df['DISEASE_filtered'].iloc[i] = ','.join(set(final_dis_list))
        df['DISEASE_filtered'].iloc[i] = df['DISEASE_filtered'].iloc[i].rstrip().lstrip()

        if (df['DISEASE_filtered'].iloc[i])!='':
            df['DISEASE_filtered'].iloc[i] = ','.join(set((df['DISEASE_filtered'].iloc[i]).split(",")))
        else:
            df['DISEASE_filtered'].iloc[i]="NA"
    except:
        pass

In [ ]:
df[['competing_trials','DISEASE_filtered']]

In [ ]:
writer = pd.ExcelWriter('C:/Users/GOWRICH1/Downloads/POS_Data_5Aug_updated.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer, sheet_name='All_data',index=False)
writer.close()